### Export a database as a dictionnary in a json file and import it back to a bw database

Pierre Jouannais

The json file is hard to read for humans because there is no space or line breaks. Feel free to modify the script to include these if you want the json file to be more readable by a human.

In [9]:

import bw2data
import bw2io
from bw2data.parameters import *
import brightway2 as bw
from json import loads, dumps
import json







In [10]:

bw.projects.set_current('Eco38_4') 
# Loading Ecoinvent
Ecoinvent = bw.Database('ecoinvent 3.8 conseq')


currentfolder = os.getcwd()

The functions

In [8]:


def remap_keys(mapping):
    
    return [{'key':k, 'value': v} for k, v in mapping.iteritems()]


def exchange_to_dict(exchange):
    
    dicti={'amount':exchange['amount'],
           'input':exchange['input'],
           'type':exchange['type']}
    
    return dicti


def export_as_dictionnary_json(bw_database,namedatabase):
    
    dict_database={}
    
    for act in bw_database:

        list_exchanges=[]
        for exc in list(act.exchanges()):
            list_exchanges.append(exchange_to_dict(exc))
        
        dicti_act={'name':act['name'],
                   'exchanges':list_exchanges,
                   'unit':act['unit'], # If we you have defined other fields in your database, you need to add it here
                   }
        
      # For instance if the activities have the field "location", this should be :  
      #  dicti_act={'name':act['name'],
      #     'exchanges':list_exchanges,
      #    'unit':act['unit'],
      #    'location':act['location'],  
     #    }
        
        
        dict_database[(namedatabase,act['code'])]=dicti_act

    ready = json.dumps({str(k): v for k, v in dict_database.items()})
    
    # open file for writing, "w" 
    f = open(namedatabase+".json","w")
    
    # write json object to file
    f.write(ready)
    
    # close file
    f.close()





def import_database_data_from_json(jsonfilepath):
      
    with open(jsonfilepath+'.json') as json_file:
        data = json.load(json_file)
      
    database = {ast.literal_eval(k): v for k, v in data.items()}
 
       
    return database   







Create a dummy database

In [14]:
act1=Ecoinvent.random()
act2=Ecoinvent.random()
act3=Ecoinvent.random()

In [15]:
db_contrib_Magic = bw.Database('db_contrib_Magic')
db_contrib_Magic.write({  
    ('db_contrib_Magic', 'magic_prod_for_sa'): {
        'name': 'Magic Prod',
        'unit': 'Magic Unit',
        'exchanges': [{                  
            'input': ('db_contrib_Magic', 'First activity'),   
            'amount': 0.2, 
            'type': 'technosphere'},  
        { 
            'input': ('db_contrib_Magic', 'Second activity'), 
            'amount': 0.2,  
            'type': 'technosphere'},    
 
        { 
            'input':  ('db_contrib_Magic', 'Third activity'), 
            'amount': -0.0000002, 
            'type': 'technosphere'},  

        {
            'input': ('db_contrib_Magic', 'magic_prod_for_sa'),
            'amount': 1.0, 
            'type': 'production'}]
    },
    ('db_contrib_Magic', 'First activity'): {
        'name': 'First activity',
        'unit': 'Strange Unit',
        'exchanges': [{
            'input': act1,  
            'amount': 1,
            'type': 'technosphere'},  
            {'input': ('db_contrib_Magic', 'First activity'),
            'amount': 1.0, 
            'type': 'production'}]},
    
    ('db_contrib_Magic', 'Second activity'): {
        'name': 'Second activity',
        'unit': 'Strange Unit',
        'exchanges': [{
            'input': act2,  
            'amount': 1, 
            'type': 'technosphere'},  
   
        {
            'input': ('db_contrib_Magic', 'Second activity'),
            'amount': 1.0,
            'type': 'production'}]},
    
    ('db_contrib_Magic', 'Third activity'): {
        'name': 'Third activity',
        'unit': 'Strange Unit',
        'exchanges': [{
            'input': act3, 
            'amount': 1, 
            'type': 'technosphere'},  
   
        {
            'input': ('db_contrib_Magic','Third activity'),
            'amount': 1.0, 
            'type': 'production'}]}})

Writing activities to SQLite3 database:
0% [####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 02/09/2023 14:01:38
  Finished: 02/09/2023 14:01:38
  Total time elapsed: 00:00:00
  CPU %: 97.70
  Memory %: 0.89


## Export the database as a light json file

In [19]:
export_as_dictionnary_json(db_contrib_Magic,'db_contrib_Magic') # It's gonna export it in your script's folder

## Import the database from the json file

In [20]:
database_dict=import_database_data_from_json('db_contrib_Magic') # with the right path, here it's in the same folder

imported_database = bw.Database("db_contrib_Magic'") # Make a BW database with the dictionnary
